In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinu

In [ ]:
%cd '/content/drive/MyDrive/'

[Errno 2] No such file or directory: '/content/drive/MyDrive/'
/content


In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: data/val.csv            
  inflating: data/train.csv          
  inflating: data/test.csv           
  inflating: data/final_val.csv      
  inflating: data/final_test.csv     
  inflating: data/final_train.csv    


# *Importing libraries*

In [2]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [3]:
import json
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer
import wandb
from transformers.integrations import WandbCallback

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")


train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])

In [5]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [6]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [7]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [8]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [9]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI'}

In [10]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI'}

In [12]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI'}

In [13]:
import pickle
with open('/content/rh-code-mixed-2.pkl', 'rb') as f:
    dict_words = pickle.load(f)

In [14]:
def corrected_preprocess(sentences, related_words):
    texts = sentences['clean_text']
    processed_texts = []

    for text in texts:
        words = text.split()
        process_text = []

        for word in words:
            # Check if the word exists in the related_words dictionary
            if related_words[word].get('corrected_word') != None :
                # Append the h_script value from related_words to the interleaved_text list
                process_text.append(related_words[word]['corrected_word'])
            else:
                # If the word is not in related_words, just append the original word
                process_text.append(word)

        # Join the list back into a string
        processed_texts.append(" ".join(process_text))

    return {"corrected_text": processed_texts}

dataset = dataset.map(
    corrected_preprocess,
    fn_kwargs={'related_words': dict_words},
    batched=True
)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [15]:
dict_words['zeher']

{'related_words': [],
 'h_script': 'ज़हर',
 'language': 'HI',
 'corrected_word': 'zehar'}

In [16]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'corrected_text': 'hindu to kuch he par tum to puri duniya may gandagi machaa rahe ho or bat rahi bhagave kii to tumhe aba ke abe bhagava hi the talwaar ke dar se salwar khol di unhone isamen tumhaari koi galati nahin he'}

# For 5000 rows dataset with corrected_text

# *Tokenization as per Hing-MBERT*

In [17]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [18]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['corrected_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [19]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460,  10114,  15694,  10269,  10261,  10248,  13055,
          10147,  10114,  32385,  10116,  23145,  14990,  11387,  13412,  34350,
         109664,  17394,  11859,  11643,  13173,  10345,  11519,  11859,  11924,
            170,  65864,  10612,  10879,  10116,  10114,  13055,  34927,  10112,
          11357,  10113,  11163,  11357,  10112,    170,  65864,  10362,  11520,
          10105,  13675,  69689,  11163,  12923,  10126,  31119,  29445,    179,
          32631,  10120,  10119,  50198,  26531,  10136,  13055,  34927,  21960,
          10116,  11252,  10116,  74288,  10325,  62137,  10115,  10261,    102,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', 'to', 'ku', '##ch', 'he', 'par', 'tu', '##m', 'to', 'pur', '##i', 'dun', '##iya', 'may', 'gan', '##dagi', 'mach', '##aa', 'ra', '##he', 'ho', 'or', 'bat', 'ra', '##hi', 'b', '##haga', '##ve', 'ki', '##i', 'to', 'tu', '##mh', '##e', 'ab', '##a', 'ke', 'ab', '##e', 'b', '##haga', '##va', 'hi', 'the', 'tal', '##waar', 'ke', 'dar', 'se', 'sal', '##war', 'k', '##hol', 'di', 'un', '##hone', 'isam', '##en', 'tu', '##mh', '##aar', '##i', 'ko', '##i', 'gala', '##ti', 'nahi', '##n', 'he', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [21]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

actual_str = dataset['train'][0]['clean_text']
input_str = dataset['train'][0]['corrected_text']
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("actual:               ", actual_str)
print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

actual:                hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai
start:                 hindu to kuch he par tum to puri duniya may gandagi machaa rahe ho or bat rahi bhagave kii to tumhe aba ke abe bhagava hi the talwaar ke dar se salwar khol di unhone isamen tumhaari koi galati nahin he
tokenize:              ['hin', '##du', 'to', 'ku', '##ch', 'he', 'par', 'tu', '##m', 'to', 'pur', '##i', 'dun', '##iya', 'may', 'gan', '##dagi', 'mach', '##aa', 'ra', '##he', 'ho', 'or', 'bat', 'ra', '##hi', 'b', '##haga', '##ve', 'ki', '##i', 'to', 'tu', '##mh', '##e', 'ab', '##a', 'ke', 'ab', '##e', 'b', '##haga', '##va', 'hi', 'the', 'tal', '##waar', 'ke', 'dar', 'se', 'sal', '##war', 'k', '##hol', 'di', 'un', '##hone', 'isam', '##en', 'tu', '##mh', '##aar', '##i', 'ko', '##i', 'gala', '##ti', 'nahi', '##n', 'he']
convert_tokens_to_ids: [19911, 11460, 1

# *Calculating Weights for Labels with Imbalanced Dataset*

In [22]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [23]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [24]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'recall_micro': recall_micro,
        'recall_macro': recall_macro,
        'recall_positive': recall_positive,
        'recall_negative': recall_negative,
        'f1_weighted': f1,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_positive': f1_positive,
        'f1_negative': f1_negative
    }

In [26]:
wandb.init(project="rh1", name="Hing_mBERT_corrected")
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_corrected_text",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"

)



trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [27]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [28]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_corrected_text/log.jsonl"))

In [29]:
# train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.686743,0.736533,0.677119,0.736533,0.736533,0.534028,0.141700,0.926357,0.688290,0.736533,0.524267,0.206490,0.842043
2,No log,0.648510,0.748286,0.790382,0.748286,0.748286,0.730607,0.696356,0.764858,0.761348,0.748286,0.697015,0.572379,0.821652
3,No log,0.500618,0.767875,0.814994,0.767875,0.767875,0.766958,0.765182,0.768734,0.780868,0.767875,0.724276,0.614634,0.833917
4,No log,0.502923,0.799216,0.816786,0.799216,0.799216,0.764199,0.696356,0.832041,0.805576,0.799216,0.744643,0.626594,0.862693
5,No log,0.505567,0.780607,0.814735,0.780607,0.780607,0.765708,0.736842,0.794574,0.791052,0.780607,0.732495,0.619048,0.845942
6,No log,0.595657,0.796278,0.812205,0.796278,0.796278,0.756748,0.680162,0.833333,0.802240,0.796278,0.739398,0.617647,0.861148
7,No log,0.693659,0.783546,0.799018,0.783546,0.783546,0.737323,0.647773,0.826873,0.789559,0.783546,0.722131,0.591497,0.852765
8,No log,0.802245,0.793340,0.804210,0.793340,0.793340,0.742405,0.643725,0.841085,0.797786,0.793340,0.730838,0.601134,0.860542


TrainOutput(global_step=384, training_loss=0.44261328379313153, metrics={'train_runtime': 138.2028, 'train_samples_per_second': 221.414, 'train_steps_per_second': 3.473, 'train_loss': 0.44261328379313153, 'epoch': 8.0})

In [30]:
mbert_results = trainer.evaluate()

In [31]:
print(mbert_results)

{'eval_loss': 0.5029232501983643, 'eval_accuracy': 0.7992164544564153, 'eval_precision_weighted': 0.8167858587454898, 'eval_recall_weighted': 0.7992164544564153, 'eval_recall_micro': 0.7992164544564153, 'eval_recall_macro': 0.7641988094864471, 'eval_recall_positive': 0.6963562753036437, 'eval_recall_negative': 0.8320413436692506, 'eval_f1_weighted': 0.8055756276743727, 'eval_f1_micro': 0.7992164544564153, 'eval_f1_macro': 0.7446431861132157, 'eval_f1_positive': 0.6265938069216758, 'eval_f1_negative': 0.8626925653047555, 'eval_runtime': 1.6149, 'eval_samples_per_second': 632.248, 'eval_steps_per_second': 39.632, 'epoch': 8.0}


# **Hing-MBERT Test results**

In [32]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [33]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 1, 1, 0])

In [34]:
test_df['5k_corrected_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [35]:
print(test_results_mbert.metrics)

{'test_loss': 0.5060051679611206, 'test_accuracy': 0.7845249755142018, 'test_precision_weighted': 0.8101574021646883, 'test_recall_weighted': 0.7845249755142018, 'test_recall_micro': 0.7845249755142018, 'test_recall_macro': 0.7572654803376957, 'test_recall_positive': 0.7044534412955465, 'test_recall_negative': 0.810077519379845, 'test_f1_weighted': 0.7931523975081209, 'test_f1_micro': 0.7845249755142018, 'test_f1_macro': 0.7317111624973722, 'test_f1_positive': 0.612676056338028, 'test_f1_negative': 0.8507462686567164, 'test_runtime': 1.6382, 'test_samples_per_second': 623.233, 'test_steps_per_second': 39.067}


# **Hing BERT Training**

In [36]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")

bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['corrected_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [37]:
## model training
wandb.init(project="rh1", name="Hing_BERT_corrected")
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_corrected_text",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=1e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_corrected_text/log.jsonl"))

eval/accuracy,▁▂▅█▆█▆▇█
eval/f1_macro,▁▆▇███▇██
eval/f1_micro,▁▂▅█▆█▆▇█
eval/f1_negative,▄▁▃█▅█▆██
eval/f1_positive,▁▇████▇██
eval/f1_weighted,▁▅▇█▇█▇██
eval/loss,▅▄▁▁▁▃▅█▁
eval/precision_weighted,▁▇████▇▇█
eval/recall_macro,▁▇████▇▇█
eval/recall_micro,▁▂▅█▆█▆▇█
eval/recall_negative,█▁▁▄▂▄▄▄▄


config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
# train the model
bert_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.696217,0.244858,0.816778,0.244858,0.244858,0.501938,1.000000,0.003876,0.100327,0.244858,0.199118,0.390514,0.007722
2,No log,0.690619,0.491675,0.687969,0.491675,0.491675,0.564113,0.704453,0.423773,0.520337,0.491675,0.479841,0.401384,0.558298
3,No log,0.679563,0.447600,0.776874,0.447600,0.447600,0.612228,0.931174,0.293282,0.446758,0.447600,0.447596,0.449219,0.445972
4,No log,0.584507,0.711068,0.786143,0.711068,0.711068,0.722599,0.744939,0.700258,0.730188,0.711068,0.670565,0.555053,0.786077
5,No log,0.548112,0.787463,0.790156,0.787463,0.787463,0.716476,0.578947,0.854005,0.788743,0.787463,0.713794,0.568588,0.858999
6,No log,0.538447,0.778648,0.799190,0.778648,0.778648,0.739607,0.663968,0.815245,0.786172,0.778648,0.720088,0.592058,0.848118
7,No log,0.545185,0.764936,0.805404,0.764936,0.764936,0.752615,0.728745,0.776486,0.777061,0.764936,0.716782,0.600000,0.833564
8,No log,0.595198,0.767875,0.801594,0.767875,0.767875,0.746284,0.704453,0.788114,0.778680,0.767875,0.716104,0.594872,0.837337
9,No log,0.622060,0.775710,0.804729,0.775710,0.775710,0.750073,0.700405,0.799742,0.785316,0.775710,0.722819,0.601739,0.843899


TrainOutput(global_step=432, training_loss=0.5253796754059968, metrics={'train_runtime': 134.3563, 'train_samples_per_second': 227.753, 'train_steps_per_second': 3.573, 'train_loss': 0.5253796754059968, 'epoch': 9.0})

In [39]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.5481120944023132, 'eval_accuracy': 0.7874632713026445, 'eval_precision_weighted': 0.790155853906205, 'eval_recall_weighted': 0.7874632713026445, 'eval_recall_micro': 0.7874632713026445, 'eval_recall_macro': 0.7164762681898544, 'eval_recall_positive': 0.5789473684210527, 'eval_recall_negative': 0.8540051679586563, 'eval_f1_weighted': 0.7887432409056723, 'eval_f1_micro': 0.7874632713026445, 'eval_f1_macro': 0.7137939097061556, 'eval_f1_positive': 0.5685884691848907, 'eval_f1_negative': 0.8589993502274205, 'eval_runtime': 1.6182, 'eval_samples_per_second': 630.929, 'eval_steps_per_second': 39.549, 'epoch': 9.0}


# **Test results for Hing-BERT**

In [40]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['5k_corrected_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.5396042466163635, 'test_accuracy': 0.781586679725759, 'test_precision_weighted': 0.7929076162457508, 'test_recall_weighted': 0.781586679725759, 'test_recall_micro': 0.781586679725759, 'test_recall_macro': 0.72638326585695, 'test_recall_positive': 0.6194331983805668, 'test_recall_negative': 0.8333333333333334, 'test_f1_weighted': 0.7862858119785879, 'test_f1_micro': 0.781586679725759, 'test_f1_macro': 0.7155303063431357, 'test_f1_positive': 0.5784499054820414, 'test_f1_negative': 0.85261070720423, 'test_runtime': 1.6386, 'test_samples_per_second': 623.107, 'test_steps_per_second': 39.059}


# **Muril cased training**

In [41]:
# wandb.init(project="rh1", name="muril_corrected")
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['corrected_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

muril_dataset = muril_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_corrected_text",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=1e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_corrected_text/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [42]:
wandb.init(project="rh1", name="muril_corrected")
muril_trainer.train()
wandb.finish()

eval/accuracy,▁▄▄▇██████
eval/f1_macro,▁▅▄▇██████
eval/f1_micro,▁▄▄▇██████
eval/f1_negative,▁▆▅▇██████
eval/f1_positive,▁▁▃▆▇████▇
eval/f1_weighted,▁▅▅▇██████
eval/loss,██▇▃▁▁▁▄▅▁
eval/precision_weighted,█▁▆▆▇▇▇▇▇▇
eval/recall_macro,▁▃▄▇▇████▇
eval/recall_micro,▁▄▄▇██████
eval/recall_negative,▁▄▃▇██▇▇██


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.693205,0.241920,0.058525,0.241920,0.241920,0.500000,1.000000,0.000000,0.094249,0.241920,0.194795,0.389590,0.000000
2,No log,0.693163,0.241920,0.058525,0.241920,0.241920,0.500000,1.000000,0.000000,0.094249,0.241920,0.194795,0.389590,0.000000
3,No log,0.693025,0.241920,0.058525,0.241920,0.241920,0.500000,1.000000,0.000000,0.094249,0.241920,0.194795,0.389590,0.000000
4,No log,0.691086,0.388834,0.772919,0.388834,0.388834,0.580360,0.951417,0.209302,0.363023,0.388834,0.385694,0.429616,0.341772
5,No log,0.672737,0.595495,0.774140,0.595495,0.595495,0.676694,0.834008,0.519380,0.621632,0.595495,0.580017,0.499394,0.660641
6,No log,0.645656,0.779628,0.779325,0.779628,0.779628,0.698904,0.542510,0.855297,0.779475,0.779628,0.699178,0.543611,0.854745
7,No log,0.627697,0.690500,0.784909,0.690500,0.690500,0.717303,0.769231,0.665375,0.712189,0.690500,0.655604,0.545977,0.765230
8,No log,0.608922,0.724780,0.792684,0.724780,0.724780,0.733021,0.748988,0.717054,0.742435,0.724780,0.683172,0.568356,0.797987
9,No log,0.607587,0.674829,0.792625,0.674829,0.674829,0.722128,0.813765,0.630491,0.698158,0.674829,0.646931,0.547684,0.746177
10,No log,0.590357,0.762977,0.788342,0.762977,0.762977,0.725136,0.651822,0.798450,0.772073,0.762977,0.703594,0.570922,0.836265


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


eval/accuracy,▁▁▁▃▆█▇▇▇█
eval/f1_macro,▁▁▁▄▆█▇█▇█
eval/f1_micro,▁▁▁▃▆█▇▇▇█
eval/f1_negative,▁▁▁▄▆█▇█▇█
eval/f1_positive,▁▁▁▃▅▇▇█▇█
eval/f1_weighted,▁▁▁▄▆█▇█▇█
eval/loss,████▇▅▄▂▂▁
eval/precision_weighted,▁▁▁███████
eval/recall_macro,▁▁▁▃▆▇████
eval/recall_micro,▁▁▁▃▆█▇▇▇█
eval/recall_negative,▁▁▁▃▅█▆▇▆█


In [43]:
wandb.init(project="rh1", name="muril_corrected")
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

wandb: Currently logged in as: shanu-dhawale11 (shanu-dhawale11-university-of-galway). Use `wandb login --relogin` to force relogin


{'eval_loss': 0.6456558704376221, 'eval_accuracy': 0.7796278158667973, 'eval_precision_weighted': 0.779324703132975, 'eval_recall_weighted': 0.7796278158667973, 'eval_recall_micro': 0.7796278158667973, 'eval_recall_macro': 0.6989036395401145, 'eval_recall_positive': 0.5425101214574899, 'eval_recall_negative': 0.8552971576227391, 'eval_f1_weighted': 0.77947544835989, 'eval_f1_micro': 0.7796278158667974, 'eval_f1_macro': 0.699177772219727, 'eval_f1_positive': 0.5436105476673428, 'eval_f1_negative': 0.8547449967721111, 'eval_runtime': 1.6232, 'eval_samples_per_second': 629.007, 'eval_steps_per_second': 39.428, 'epoch': 10.0}


In [44]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['5k_corrected_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

{'test_loss': 0.6488091945648193, 'test_accuracy': 0.7590597453476984, 'test_precision_weighted': 0.7637966091786141, 'test_recall_weighted': 0.7590597453476984, 'test_recall_micro': 0.7590597453476984, 'test_recall_macro': 0.6812028580694431, 'test_recall_positive': 0.5303643724696356, 'test_recall_negative': 0.8320413436692506, 'test_f1_weighted': 0.7612803216149231, 'test_f1_micro': 0.7590597453476984, 'test_f1_macro': 0.6776914864129597, 'test_f1_positive': 0.515748031496063, 'test_f1_negative': 0.8396349413298565, 'test_runtime': 1.6396, 'test_samples_per_second': 622.73, 'test_steps_per_second': 39.035}


# XLM-R-Base Training

In [45]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [46]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['corrected_text'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","corrected_text","language_tags"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_corrected_text",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=1e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_corrected_text/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
wandb.init(project="rh1", name="xlmr_corrected")
xlmr_trainer.train()
wandb.finish()

eval/accuracy,▁
eval/f1_macro,▁
eval/f1_micro,▁
eval/f1_negative,▁
eval/f1_positive,▁
eval/f1_weighted,▁
eval/loss,▁
eval/precision_weighted,▁
eval/recall_macro,▁
eval/recall_micro,▁
eval/recall_negative,▁


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.711848,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
2,No log,0.697810,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
3,No log,0.683548,0.749265,0.676979,0.749265,0.749265,0.518995,0.072874,0.965116,0.677010,0.749265,0.488501,0.123288,0.853714
4,No log,0.673255,0.761019,0.716279,0.761019,0.761019,0.551557,0.145749,0.957364,0.706033,0.761019,0.543240,0.227848,0.858633
5,No log,0.648269,0.766895,0.736975,0.766895,0.766895,0.603673,0.287449,0.919897,0.739924,0.766895,0.615242,0.373684,0.856799
6,No log,0.617927,0.764936,0.758044,0.764936,0.764936,0.664404,0.469636,0.859173,0.761105,0.764936,0.669330,0.491525,0.847134
7,No log,0.593934,0.762977,0.776102,0.762977,0.762977,0.703083,0.587045,0.819121,0.768460,0.762977,0.692424,0.545113,0.839735
8,No log,0.588070,0.726738,0.783490,0.726738,0.726738,0.720530,0.708502,0.732558,0.743009,0.726738,0.679493,0.556439,0.802548
9,No log,0.598276,0.775710,0.781136,0.775710,0.775710,0.705968,0.570850,0.841085,0.778196,0.775710,0.701142,0.551859,0.850425
10,No log,0.596616,0.767875,0.781161,0.767875,0.767875,0.710448,0.599190,0.821705,0.773367,0.767875,0.699145,0.555347,0.842942


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


eval/accuracy,▅▅▄▆▇▆▆▁█▇
eval/f1_macro,▁▁▂▄▆▇█▇██
eval/f1_micro,▅▅▄▆▇▆▆▁█▇
eval/f1_negative,██▇█▇▆▅▁▇▆
eval/f1_positive,▁▁▃▄▆▇████
eval/f1_weighted,▁▁▂▄▆▇▇▆██
eval/loss,█▇▆▆▄▃▁▁▂▁
eval/precision_weighted,▁▁▄▆▆▇████
eval/recall_macro,▁▁▂▃▄▆▇███
eval/recall_micro,▅▅▄▆▇▆▆▁█▇
eval/recall_negative,██▇▇▆▄▃▁▄▃


In [48]:
wandb.init(project="rh1", name="xlmr_corrected")
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.5982757806777954, 'eval_accuracy': 0.7757100881488737, 'eval_precision_weighted': 0.7811361463311529, 'eval_recall_weighted': 0.7757100881488737, 'eval_recall_micro': 0.7757100881488737, 'eval_recall_macro': 0.7059677368734896, 'eval_recall_positive': 0.5708502024291497, 'eval_recall_negative': 0.8410852713178295, 'eval_f1_weighted': 0.7781956967719027, 'eval_f1_micro': 0.7757100881488737, 'eval_f1_macro': 0.7011418294579985, 'eval_f1_positive': 0.5518590998043053, 'eval_f1_negative': 0.8504245591116917, 'eval_runtime': 1.6156, 'eval_samples_per_second': 631.978, 'eval_steps_per_second': 39.615, 'epoch': 10.0}


In [49]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['predicted_labels_5k_corrected_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 0.5775119066238403, 'test_accuracy': 0.7835455435847208, 'test_precision_weighted': 0.7920792889381041, 'test_recall_weighted': 0.7835455435847208, 'test_recall_micro': 0.7835455435847208, 'test_recall_macro': 0.7235403655232296, 'test_recall_positive': 0.6072874493927125, 'test_recall_negative': 0.8397932816537468, 'test_f1_weighted': 0.7872330549093574, 'test_f1_micro': 0.7835455435847208, 'test_f1_macro': 0.7152582968321931, 'test_f1_positive': 0.5758157389635316, 'test_f1_negative': 0.8547008547008547, 'test_runtime': 1.6569, 'test_samples_per_second': 616.213, 'test_steps_per_second': 38.626}


In [50]:
wandb.finish()

eval/accuracy,▁
eval/f1_macro,▁
eval/f1_micro,▁
eval/f1_negative,▁
eval/f1_positive,▁
eval/f1_weighted,▁
eval/loss,▁
eval/precision_weighted,▁
eval/recall_macro,▁
eval/recall_micro,▁
eval/recall_negative,▁


In [51]:
test_df.to_csv('5k_corrected_v1.csv',index=False, encoding='utf-8')